# Solution computation from models

This notebook demonstrates the calculation of a solution to a model:
- Creating Moon models and mesh (M1, M2, M3, weber_core_smooth, weber_core)
- Source and receiver definition with computation of travel times
- Inversion of data using LU factorization

In [ ]:
import numpy as np
from sensray import PlanetModel
from fwdop import GFwdOp, make_scalar_field
from pygeoinf.linear_solvers import LUSolver, CholeskySolver

## 1. Creation of Moon model and mesh

Creating a model and mesh for ray tracing

### Creating model

Creating a moon model from information stored in .nd file
- User-defined model (model_name), mesh size, and inner core mesh size
- Model creation of PlanetModel class

In [29]:
# Load model and create mesh
model_name = "M1"
mesh_size_km = 1000

# Create mesh and save if not exist, otherwise load existing
mesh_path = "M1_mesh"

# Load model and create mesh
model = PlanetModel.from_standard_model('M1')

### Creating layered tetrahedral mesh

Uses a layered tetrahedral mesh to control resolution across discontinuities
- User-defined mesh name
- Radii and H_layers gathered from the model properties
- Populate mesh properties with vp, vs, rho from the file (also gathered from model)

In [30]:
try:
    model.create_mesh(from_file=mesh_path)
    print(f"Loaded existing mesh from {mesh_path}")
except FileNotFoundError:
    print("Creating new mesh...")
    radii = model.get_discontinuities()
    H_layers = [1000, 600]
    model.create_mesh(mesh_size_km=mesh_size_km, radii=radii, H_layers=H_layers)
    model.mesh.populate_properties(model.get_info()["properties"])
    model.mesh.save(f"{model_name}_mesh")  # Save mesh to VT
print(f"Created mesh: {model.mesh.mesh.n_cells} cells")

Loaded mesh from M1_mesh.vtu
Loaded metadata: 9975 cells, 2258 points
Loaded existing mesh from M1_mesh
Created mesh: 9975 cells


## 2. Source-Receiver geometry and travel time computation

Set up realistic earthquake-station-phase ray generation

In [31]:
# testing with one source-receiver pair - produces same matrix as with other demos
source_lat, source_lon, source_depth = 0.0, 0.0, 150.0  # Equator, 150 km depth
receiver_lat, receiver_lon = 30.0, 45.0  # Surface station
srp = [((source_lat, source_lon, source_depth), (receiver_lat, receiver_lon), ["P"])]

def get_rays(srp):
    '''
    srp: list of tuples (source, receiver, phases)
    where source = (lat, lon, depth), receiver = (lat, lon), phases = [phase1, phase2, ...]
    returns array of (source, receiver, ray) for each ray
    '''
    srr_list = []
    for (source, receiver, phases) in srp:
        rays = model.taupy_model.get_ray_paths_geo(
            source_depth_in_km=source[2],
            source_latitude_in_deg=source[0],
            source_longitude_in_deg=source[1],
            receiver_latitude_in_deg=receiver[0],
            receiver_longitude_in_deg=receiver[1],
            phase_list=phases,
        )
        for ray in rays:
            srr_list.append((source, receiver, ray))

    return np.array(srr_list, dtype=object)


print("Compute ray for each source-receiver-phase combination...")
srr = get_rays(srp)

Compute ray for each source-receiver-phase combination...
Building obspy.taup model for '/home/matth/Masters-Project/SensRay/sensray/models/M1.nd' ...
filename = /home/matth/Masters-Project/SensRay/sensray/models/M1.nd
Done reading velocity model.
Radius of model . is 1737.1
Using parameters provided in TauP_config.ini (or defaults if not) to call SlownessModel...
Parameters are:
taup.create.min_delta_p = 0.1 sec / radian
taup.create.max_delta_p = 11.0 sec / radian
taup.create.max_depth_interval = 115.0 kilometers
taup.create.max_range_interval = 0.04363323129985824 degrees
taup.create.max_interp_error = 0.05 seconds
taup.create.allow_inner_core_s = True
Slow model  643 P layers,747 S layers
Done calculating Tau branches.
Done Saving /tmp/M1.npz
Method run is done, but not necessarily successful.


### Calculate travel times

Compute sensitivity kernels and project functions on the mesh

Calculate travel times for the function as in other demos

In [32]:
print("Calculate sensitivity kernels and sparse G matrix...")
G = GFwdOp(model, srr[:,2])
G.save_matrix("M1_sp_G_mat.npz")

# Generating a scalar field from functions in R, T
print(f"Scalar field generation...")

functions = {
    "simple": {"R": lambda r: np.ones_like(r), "T": lambda theta, phi: np.ones_like(theta)},
    "complex": {"R": lambda r: r**2 * np.exp(-r/100000), "T": lambda theta, phi: np.cos(theta)},
    "harmonic": {"R": lambda r: 0.1 * model.get_property_at_radius(radius=r, property_name="vp"), "T": lambda theta, phi: 0.5 * np.sqrt(5 / np.pi) * (3 * np.cos(theta) ** 2 - 1)},
}

func = "harmonic"
f = make_scalar_field(functions[func]["R"], functions[func]["T"])

print("Dv projection onto mesh...")
model.mesh.project_function_on_mesh(f, property_name="dv")

print("Time travel computation...")
travel_times = G(model.mesh.mesh.cell_data["dv"])
print(travel_times)

Calculate sensitivity kernels and sparse G matrix...
Stored sensitivity kernel as cell data: 'K_P_vp'
P Kernel Matrix shape: (1, 9975), nnz: 24
Scalar field generation...
Dv projection onto mesh...
Time travel computation...
[8.34158499]


## 3. Compute inverse operator

- Test adjoint operation
- Compute and test lambda (lambda = GG^T)
- Define and test solver
- Compute and test tilde_M (tilde_M = G_dagger(d) where d=G(m))


### Adjoint Operation

Test adjoint - Correct if G.adjoint(v) = first kernel (G._K)

In [33]:
kernel_choice = 0  # desired kernel
v = np.zeros(G._K.shape[0])
v[kernel_choice] = 1
print(f"Test Adjoint Success: {np.allclose(G.adjoint(v), G._K.getrow(kernel_choice).toarray().ravel())}")

Test Adjoint Success: True


### Lambda Operation

Compute lambda from G and G.adjoint (GG^T)

Test lambda - Correct if dense matrix is symmetric

In [34]:
# Compute lambda from G and G.adjoint (GG^T)
lambda_val = (G@G.adjoint)
# Test lambda - Correct if dense matrix is symmetric
print(f"Test lambda Success: {np.allclose(lambda_val.matrix(dense=True), lambda_val.matrix(dense=True).T)}")

Test lambda Success: True


### Define and test solver

Define what type of linear solver you want to use (LU, Cholesky, etc)

Apply to lambda to return lambda inverse

Test solver - Shouldn't raise any kinds of errors

In [ ]:
# Test solver - Shouldn't raise an error of any kind
solver = LUSolver()
lambda_inv = solver(lambda_val)

### Compute cell inverse and M_tilde

Compute the cell inverse (G_dagger) from G adjoint and lambda inverse

Compute solution M_tilde (vector of length Nd) from the cell inverse applied to the travel times

Add the solution to the cell data

In [ ]:
# G_dagger - Should compute cell inverse
G_dagger = G.adjoint@lambda_inv

# M_tilde - G_dagger(d) where d is G applied onto the model. Dims N voxels
M_tilde = G_dagger(travel_times)

model.mesh.mesh.cell_data["solution"] = M_tilde

# Test - For a single ray (getting 0th row) M_tilde = kernel * G(m) / lambda
print(f"Single ray inverse calculation: {np.allclose(M_tilde, (G.matrix(dense=True) * travel_times / lambda_val.matrix(dense=True))[0])}")

True
